#### Flujo del programa:
     main_program(path_files, path_organized_files)
         df = df_without_payroll(path_files, path_organized_files)
             xml_file_names = getting_xml_files(path_files)
             dic, no_folio = main(xml_file_names, path_files)
                 client_name, folio_number, letter = getting_name_folio(file, path_files)
             * generate_excel(no_folio, '/no_folio')
         organized_files(df, path_files, path_organized_files)
             xml_files_number_BEFORE = getting_xml_files(path_files)
             xml_files_number_AFTER = getting_xml_files(path_files)
             xml_organized_files_number_AFTER = getting_xml_files(path_organized_files)
             * generate_excel(fail_data_dic, '/fail_data_organized_files')
		

#### Importamos librerías necesarias

In [1]:
import os
import glob
import xml.etree.ElementTree as ET
import pandas as pd
import shutil
from zipfile import ZipFile

#### Funciones a utilizar

In [2]:
#Esta función devuelve un diccionario y una lista. El diccionario guarda 4 datos que caracterizan cada archivo XML. La lista
#guarda los nombres de los archivos que no contienen folio.
def main(xml_file_names, path_files):
    dic = {"client_name": [], "folio_number": [], "name_file": [], "letter": []}
    no_folio = []
    
    for file in xml_file_names:
        try:
            #Se obtienen los 4 datos (en caso de que el folio exista)
            client_name, folio_number, letter = getting_name_folio(file, path_files)
            dic["client_name"].append(client_name)
            dic["folio_number"].append(folio_number)
            dic["name_file"].append(file)
            dic["letter"].append(letter)
        except:
            no_folio.append(file)

    return dic, no_folio

In [3]:
#Esta función entrega una lista con todos los nombres de los arvhicos XML bajados de la página del SAT
def getting_xml_files(path_files):
    #Se define la ruta del directorio de los archivos XML
    directory_path = os.path.expanduser(path_files)

    #Se usa glob para filtrar achivos que sean solo XML
    xml_files = glob.glob(os.path.join(directory_path, '*.xml'))

    #Se obtienen los nombres de cada uno de los archivos XML y se guardan en la lsita "xml_file_names"
    xml_file_names = [os.path.basename(file) for file in xml_files]
    
    return xml_file_names

In [4]:
#Esta función entrada a cada archivo XML y trata de recuperar los 4 datos que caraterizan al archivo: nombre del cliente, folio,
#nombre original del archivo y letra (tipo de comprobante).
def getting_name_folio(file, path_files):
    receptor = "GRUPO TECNICO IMPRESOR" #GTI
    
    #Se construye el arbol del archivo XML
    path = path_files + '/' + f"{file}"
    tree = ET.parse(path)
    root = tree.getroot()

    try:
        #Se recupera el folio del XML desde la raíz. Si el folio no existe se sale del "try" y se entra al "except".
        folio_number = root.attrib['Folio']
        
        #Se recupera el tipo de comprobante desde la raíz (siempre existe).
        letter = root.attrib['TipoDeComprobante']
        
        #Se navega en los hijos de las rutas de árbol.
        for child in root:
            #Se busca el hijo con terminación Receptor.
            if child.tag.endswith('Receptor'):
                #Se verifica si el atributo Nombre del hijo Receptor no es GTI. En ese caso GTI es emisor.
                if child.attrib['Nombre'] != receptor:
                    #Se guarda el nombre del cliente que no es GTI.
                    client_name = child.attrib['Nombre']
                else:
                    #Se busca al hijo con terminación Emisor y se recupera el nombre. En este caso GTI es receptor.
                    for child in root:
                        if child.tag.endswith('Emisor'):
                            #Se guarda el nombre del cliente que no es GTI.
                            client_name = child.attrib['Nombre']
                        else:
                            pass
            else:
                pass
        
        return client_name, folio_number, letter
    
    except:
        pass

In [5]:
#Esta función crea y devuelve un Dataframe que deshecha los XML que no contengan folio y que sean de nómina.
def df_without_payroll(path_files, path_organized_files):
    #Nombre de cliente no deseado.
    no_client = 'GRUPO TECNICO IMPRESOR'
    
    #Recuperamos lista con nombres de los arvhicos XML provenientes de la página del SAT.
    xml_file_names = getting_xml_files(path_files)

    #Se recupera un diccionario que contiene 4 datos que caracterizan cada XML y una lista con los nombres de los XML que
    #no contienen folio (los nombres son los asignados por el SAT).
    dic, no_folio = main(xml_file_names, path_files)
    
    #Si hay XML que no tienen folio se crea un csv con la lista "no_folio"
    if len(no_folio) != 0:
        generate_excel(no_folio, '/no_folio')
    else:
        pass
    
    #Se crea un DataFrame con los XML que sí tienen folio
    df = pd.DataFrame.from_dict(dic) 
    
    #Se obtiene el número de filas con el atributo 'N' en la columna 'letter'.
    u = df[df['letter'] == 'N'].shape[0]
    
    print(f'Archivos XML totales examinados: {len(xml_file_names)}')
    print(f'Archivos XML sin folio: {len(no_folio)}') 
    print(f'Archivos XML con folio: {len(xml_file_names)-len(no_folio)}')
    print(f'Archivos XML de nómina: {u}\n')
    
    #El df se actualiza quitando las filas con atributos 'N' de la columna 'letter'
    df = df[df['letter'] != "N"]
    
    #Se verifica que el df final tenga como número filas el resultado de la operación:
    #"número de XML totales" - "número de archivos sin folio" - "número de archivos de nómina".
    if df.shape[0] != (len(xml_file_names)-len(no_folio)-u):
        print("ERROR: número de archivos con folio no esperado. No se devuelve Dataframe.")
        
    #Este error se da si el XML tiene una estructura distinta que no tenga los hijos 'Receptor' o 'Emisor'; este error
    #puede provenir de la función getting_name_folio().
    elif (df[df['client_name'] == no_client]).shape[0] != 0:
        print("ERROR: GTI no se eliminó del nombre, checar función getting_name_folio(). No se devuelve Dataframe.")
        
    else:
        print("DataFrame generado con EXITO.")
        return df

In [6]:
def organized_files(df, path_files, path_organized_files):
    fail_data_dic = {'columns':["client_name", "folio_number", "name_file", "letter"], 'rows':[]}
    
    xml_files_number_BEFORE = getting_xml_files(path_files)
    
    print(f"Archivos XML en carpeta files: {len(xml_files_number_BEFORE)}")
    print(f"Archivos XML en carpeta organized_files: {len(os.listdir(path_organized_files))}")
    print(f"Archivos XML a renombrar: {df.shape[0]}\n")
    
    #DafaFrame a diccionario usando 'split': datos de una fila del df se guardan en una lista bajo la llave 'data'.
    dic = df.to_dict('split')
    
    #Seleccionamos la llave 'data' del diccionario.
    data_list = dic['data']
    
    #Se itera sobre cada uno de los datos que caracterizan a cada XML, i.e., sobre cada archivo XML.
    for i,data in enumerate(data_list):
        #Se quitan las comillas (en caso de tenerlas) al nombre del cliente.
        n = data[0].replace('"','')
        
        #Se le da la estructura al nombre del archivo final XML.
        rename_file = f'{data[1]} {data[3]}-{n} {data[2]}' + '.xml'
        
        #Se forma la ruta del archivo XML con el nombre dado por el SAT.
        source_path = path_files + '/' + data[2]
        
        #Se forma la ruta del archivo XML final; este va a al directorio organized_files.
        destination_path = path_organized_files + '/' + rename_file
        
        try:
            #Se mueven los archivos modificados de la ruta fuente a la ruta destino.
            shutil.move(source_path, destination_path)
        except:
            #Este error saldrá si el procesamiento de los datos tiene un error no contemplado que no permita un 
            #"destination_path" válido.
            print(f'ERROR: datos no compatibles con la estructura del programa en la iteración {i}.')
            fail_data_dic['rows'].append(data)
     
    xml_files_number_AFTER = getting_xml_files(path_files)
    xml_organized_files_number_AFTER = getting_xml_files(path_organized_files)
    
    #El número de columnas del df tiene que ser el mismo número de archivos en la carpeta organized_files
    if df.shape[0] != len(os.listdir(path_organized_files)):
        print('------'*10)
        print('ERROR: hubo datos no compatibles; checar EXCEL generado: fail_data_organized_files.\n')
        print(f"Archivos XML en carpeta files: {len(xml_files_number_AFTER)}")
        print(f"Archivos XML en carpeta organized_files: {len(xml_organized_files_number_AFTER)}\n")
        generate_excel(fail_data_dic, '/fail_data_organized_files')
    else:
        print('------'*10)
        print(f"Archivos XML en carpeta files: {len(xml_files_number_AFTER)}")
        print(f"Archivos XML en carpeta organized_files: {len(xml_organized_files_number_AFTER)}\n")
        
        print("Organización de archivos XML relizada con EXITO")

In [7]:
#Esta función crea cvs's dependiendo del requerimiento.
def generate_excel(ddata, type_data):
    '''ddata: es lista en caso de que type_data='/no_folio' o diccionario en caso de que type_data='/fail_data_organized_files'
       type_data: string que permite crear un csv específico'''
    
    #Ruta del csv; hay que cambiar dependiendo de la localización de los archivos y del programa.
    path_excel = "C:/Users/mauvp/Desktop/XML_FILES/excel"
    
    if type_data == '/fail_data_organized_files':
        file_path = path_excel + type_data + '.csv'
        
        df_fail = pd.DataFrame(data=ddata['rows'], columns=ddata['columns'])
        
        df_fail.to_csv(path_or_buf=file_path, index=False, encoding='utf-8-sig')
            
    #Creación del csv en caso de que haya archivos que no tengan folio
    elif type_data == '/no_folio':
        dic = {'columns':['name_file_sat'], 'rows':ddata}
        
        file_path = path_excel + type_data + '.csv'
        
        df_no_folio = pd.DataFrame(data=dic['rows'], columns=dic['columns'])
        
        df_no_folio.to_csv(path_or_buf=file_path, index=False, encoding='utf-8-sig')

In [8]:
def main_program(path_files, path_organized_files):
    try:
        #Se obtiene el DataFrame que no contiene XML sin folio y XML de nómina
        df = df_without_payroll(path_files, path_organized_files)
    except:
        print("FIN DEL PROGRAMA. No se ubtuvo el DataFrame.")
        pass
    else:
        print('------'*10)
        
        #Se organizan los archivos con la siguiente estructura:
        #"Número de folio" "Tipo de comprobante"-"Nombre empresa (no GTI)" "Nombre archivo dado por el SAT"
        organized_files(df, path_files, path_organized_files)
        print("FIN DEL PROGRAMA.")

#### Programa y organización de archivos

In [11]:
# Las carpetas a las que referencian estas rutas ya deben existir en el directorio
path_files = 'C:/Users/mauvp/Desktop/XML_FILES/files'
path_organized_files = 'C:/Users/mauvp/Desktop/XML_FILES/organized_files'

In [12]:
main_program(path_files, path_organized_files)

Archivos totales examinados: 387
Archivos sin folio: 0
Archivos con folio: 387
Archivos de nómina: 279

DataFrame generado con EXITO.
------------------------------------------------------------
Archivos en carpeta files: 388
Archivos en carpeta organized_files: 0
Carpetas a renombrar: 108



Archivos en carpeta files: 280
Archivos en carpeta organized_files: 108

Organización de archivos xml relizada con EXITO
FIN DEL PROGRAMA.
